# ESG with AI Data Structure

We want a multi-indexed data set that looks like this to run our analysis:

In [1]:
import pandas as pd
import numpy as np

stocks = ['A','B','C']
dates = ['2020-01-31','2020-02-29','2020-03-31']

idx = pd.MultiIndex.from_product([dates,stocks], names=["Date","Stock"])

df = pd.DataFrame(index=idx, columns=['Return Data','Equity Data','ESG Data','Signals','Wgt'])

df

Return Data Equity Data ESG Data Signals  Wgt
Date       Stock                                              
2020-01-31 A             NaN         NaN      NaN     NaN  NaN
           B             NaN         NaN      NaN     NaN  NaN
           C             NaN         NaN      NaN     NaN  NaN
2020-02-29 A             NaN         NaN      NaN     NaN  NaN
           B             NaN         NaN      NaN     NaN  NaN
           C             NaN         NaN      NaN     NaN  NaN
2020-03-31 A             NaN         NaN      NaN     NaN  NaN
           B             NaN         NaN      NaN     NaN  NaN
           C             NaN         NaN      NaN     NaN  NaN

## Bloomberg ESG

Our Bloomberg data comes out in a different format, with the equity name in the column names. We really just need to rename columns, and then set a multi-index to match our above DF.

In [4]:
# Import Dataset Using Pandas
loc = "https://github.com/mhall-simon/ESG-Investing/blob/main/test-data/grid1_s4wvcoub.xlsx?raw=true"
df = pd.read_excel(loc,index_col=0,parse_dates=True)

new_cols = []
for col in df.columns:
    idx = col.split(" \n")[0]
    new_cols.append(idx)
new_cols = pd.Index(new_cols)

equity_name = df.columns[0].split("(")[-1][:-1]

dates = df.index.get_level_values(0)

idx = pd.MultiIndex.from_product([dates,[equity_name]], names=["Date","Equity"])

df = pd.DataFrame(df.values, columns=new_cols, index=idx)

df.head()

,,Waste Generated per Sales,Pct Independent Directors,Employee Turnover Pct,Pct Water Recycled,Board Size,Pct Employees Unionized,Board Meeting Attendance Pct,Total GHG CO2 Emissions Intensity per Sales,Lost Time Incident Rate - Employees,Percent of Board Members that are Women,Energy Intensity per Sales,Board Average Age,Pct Women in Workforce,Water Intensity per Sales,Board Age Range,Women Management to Employees Ratio
Date,Equity,,,,,,,,,,,,,,,,
2021-02-11,TSLA US Equity,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,22.2222,NaN,55.2222,NaN,NaN,28.0,NaN
2021-01-31,TSLA US Equity,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,22.2222,NaN,55.1111,NaN,NaN,28.0,NaN
2020-12-31,TSLA US Equity,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,22.2222,NaN,55.0000,NaN,NaN,28.0,NaN
2020-11-30,TSLA US Equity,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,22.2222,NaN,54.8889,NaN,NaN,29.0,NaN
2020-10-31,TSLA US Equity,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,22.2222,NaN,54.8889,NaN,NaN,29.0,NaN


## Let's wrap it in a function now

In [5]:
def import_bloomberg_esg(loc):
    """
    Takes raw Excel output from Bloomberg Terminal & Converts into Proper Data Structure for Bloomberg ESG Stats.
    
    Inputs:
    loc - link to Excel file
    
    Outputs:
    DataFrame - MultiIndexed by Month-End Date & Equity Name
    """
    df = pd.read_excel(loc,index_col=0,parse_dates=True)
    
    # Strip Column Names Into First Part, Removing Equity Name & \n
    new_cols = []
    for col in df.columns:
        idx = col.split(" \n")[0]
        new_cols.append(idx)
    new_cols = pd.Index(new_cols)
    
    # Note Equity Name
    equity_name = df.columns[0].split("(")[-1][:-1]
    
    # Strip Dates From Original Index
    dates = df.index.get_level_values(0)
    
    # Create Multi-Index with Date & Equity Name
    idx = pd.MultiIndex.from_product([dates,[equity_name]], names=["Date","Equity"])
    
    # Return Updated DataFrame in Proper Format
    return pd.DataFrame(df.values, columns=new_cols, index=idx)

## Test Case #1

S&P Capital IQ ESG Rankings + Return / Mkt Cap / Properties

*Need to Add Some Fields for Value Signal / Mkt Cap*

In [13]:
loc = "https://github.com/mhall-simon/ESG-Investing/blob/main/test-data/grid1_zeapet2i.xlsx?raw=true"

import_bloomberg_esg(loc).head(3)

,,Last Price,S&P Global ESG Rank,S&P Global Governance & Economic Dimension Rank,S&P Global Environmental Dimension Rank,S&P Global Social Dimension Rank
Date,Equity,,,,,
2021-02-11,TSLA US Equity,808.159973,NaN,NaN,NaN,NaN
2021-01-31,TSLA US Equity,793.530000,NaN,NaN,NaN,NaN
2020-12-31,TSLA US Equity,705.670000,NaN,NaN,NaN,NaN


## Test Case #2

Bloomberg ESG Data

In [12]:
loc = "https://github.com/mhall-simon/ESG-Investing/blob/main/test-data/grid1_s4wvcoub.xlsx?raw=true"

import_bloomberg_esg(loc).head(3)

,,Waste Generated per Sales,Pct Independent Directors,Employee Turnover Pct,Pct Water Recycled,Board Size,Pct Employees Unionized,Board Meeting Attendance Pct,Total GHG CO2 Emissions Intensity per Sales,Lost Time Incident Rate - Employees,Percent of Board Members that are Women,Energy Intensity per Sales,Board Average Age,Pct Women in Workforce,Water Intensity per Sales,Board Age Range,Women Management to Employees Ratio
Date,Equity,,,,,,,,,,,,,,,,
2021-02-11,TSLA US Equity,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,22.2222,NaN,55.2222,NaN,NaN,28.0,NaN
2021-01-31,TSLA US Equity,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,22.2222,NaN,55.1111,NaN,NaN,28.0,NaN
2020-12-31,TSLA US Equity,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,22.2222,NaN,55.0000,NaN,NaN,28.0,NaN


## Test Case #3

Sustainalytics ESG Scores

In [11]:
loc = "https://github.com/mhall-simon/ESG-Investing/blob/main/test-data/grid1_d1kfg5mb.xlsx?raw=true"

import_bloomberg_esg(loc).head(3)

,,Sustainalytics Rank,Sustainalytics Environment Percentile,Sustainalytics Social Percentile,Sustainalytics Governance Percentile,Sustainalytics Environment Controversies,Sustainalytics Social Controversies,Sustainalytics Governance Controversies
Date,Equity,,,,,,,
2021-01-31,TSLA US Equity,26.315789,82.631579,1.052632,5.263158,NaN,NaN,NaN
2020-12-31,TSLA US Equity,26.315789,82.631579,1.052632,5.263158,NaN,NaN,NaN
2020-11-30,TSLA US Equity,25.925926,82.539683,1.058201,4.761905,NaN,NaN,NaN
